<a href="https://colab.research.google.com/github/ranjith-p/Coursera_Capstone/blob/main/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

In [6]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
